In [1]:
import org.zouzias.spark.lucenerdd._

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.12:4042
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1653051987468)
SparkSession available as 'spark'


import org.zouzias.spark.lucenerdd._


In [118]:
val table = spark.read.parquet("/tmp/phone-calls/VendorContacts").selectExpr("ObjectID as ID", "Phone")

table: org.apache.spark.sql.DataFrame = [ID: int, Phone: string]


In [112]:
import org.apache.spark.sql.Row

// +18287377552
def searchQuery: Row => String = {
    case row => {
        val search = Seq("Phone")
            .map(key => key -> row.get(row.fieldIndex(key)))
            .filter(_._2 != null)
            .map { case (key, value) =>
                val string = value
                    .toString
                    .replaceAll("[^0-9]", "")
                    .split("")
                    .mkString("-")
                (key, string)
            }
            .filter(_._2.nonEmpty)
            .map { case (key, value) => s"(${key}:(${value}))" }
            .mkString("-")

        if (search.nonEmpty) search else "*:*"
    }
}

// searchQuery(table.sample(0.01).first)

import org.apache.spark.sql.Row
searchQuery: org.apache.spark.sql.Row => String


In [121]:
val sample = Seq("8287377552").toDF("Phone")
// val sample = Seq("1 828 737 5752").toDF("Phone")

sample: org.apache.spark.sql.DataFrame = [Phone: string]


In [108]:
val table = Seq("1 828 737 7552").toDF("Phone")

table: org.apache.spark.sql.DataFrame = [Phone: string]


In [113]:
searchQuery(sample.first)

res66: String = (Phone:(1-8-2-8-7-3-7-5-7-5-2))


In [114]:
searchQuery(table.first)

res67: String = (Phone:(1-8-2-8-7-3-7-7-5-5-2))


In [122]:
val luceneRDD = LuceneRDD(table)

val results = luceneRDD.linkDataFrame(sample, searchQuery, 8)

luceneRDD: org.zouzias.spark.lucenerdd.LuceneRDD[org.apache.spark.sql.Row] = LuceneRDD[412] at RDD at LuceneRDD.scala:54
results: org.apache.spark.rdd.RDD[(org.apache.spark.sql.Row, Array[org.apache.spark.sql.Row])] = MapPartitionsRDD[427] at map at LuceneRDD.scala:248


In [123]:
// results.filter(_._2.nonEmpty).take(8)

results.take(1)

// _table.show()

res70: Array[(org.apache.spark.sql.Row, Array[org.apache.spark.sql.Row])] = Array(([8287377552],Array([8007968707, opt1, 2, 7,31410184,0,11.4012165,128156], [8,29617332,0,11.135309,109245], [8007435637 opt 7,31339053,0,9.047127,221824], [8008888018 opt 7,34904976,0,9.047127,140100], [(310) 547-4973 ext 7,34513688,0,6.329067,241409], [8003321375 opt 5,29603247,0,5.921568,269237], [18007332383 Opt. 5,29542997,0,5.921568,36788], [4126427600 option 2,29524908,0,4.802253,21017])))
